# Prepare reference DEM

In [ ]:
import os,glob
import xdem
import subprocess
import matplotlib.pyplot as plt

In [ ]:
data_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites'
site_names = ['MCS', 'JacksonPeak', 'Banner']

for site_name in site_names:
    print(f"\n{site_name}")
    if site_name=='JacksonPeak':
        lidar_fn = os.path.join(data_dir, site_name, 'refdem', 'USGS_LPC_ID_FEMAHQ_2018_D18_merged_filtered.tif')
    else:
        lidar_fn = os.path.join(data_dir, site_name, 'refdem', f'{site_name}_REFDEM_WGS84.tif')
    copdem_fn = glob.glob(os.path.join(data_dir, site_name, 'refdem', 
                                       f'{site_name}_COPDEM_clip_buffer*_WGS84_ellipsoid_UTM.tif'))[0]
    
    # Fill gaps in DEMs
    def fill_dem_gaps(dem_fn):
        out_fn = os.path.splitext(dem_fn)[0] + '_filled.tif'
        if not os.path.exists(out_fn):
            print('Filling gaps in DEMs')
            # construct command
            cmd = ['gdal_fillnodata.py', 
                   '-md', '10',
                   '-of', 'GTiff',
                   dem_fn, out_fn]
            # run command
            out = subprocess.run(cmd, shell=False, capture_output=True)
            print(out)
        else:
            print('Filled DEM already exists, skipping.')
        return out_fn
    lidar_fill_fn = fill_dem_gaps(lidar_fn)
    copdem_fill_fn = fill_dem_gaps(copdem_fn)
    
    # Coregister COPDEM to the lidar
    copdem_fill_coreg_fn = os.path.splitext(copdem_fill_fn)[0] + '_coreg.tif'
    if not os.path.exists(copdem_fill_coreg_fn):
        print('Coregistering using the Nuth and Kaab approach')
        lidar = xdem.DEM(lidar_fill_fn)
        copdem = xdem.DEM(copdem_fill_fn).reproject(lidar)
        nk = xdem.coreg.NuthKaab().fit(lidar, copdem)
        print(nk.meta)
        copdem_nk = nk.apply(copdem)
        copdem_nk.save(copdem_fill_coreg_fn)
        print('Coregistered COPDEM saved to file:', copdem_fill_coreg_fn)
    else:
        print('Coregistered COPDEM already exists, skipping.')
        
    # Merge DEMs: Overlay the COPDEM with the lidar DEM
    merged_dem_fn = os.path.join(os.path.dirname(copdem_fn), f'{site_name}_refdem_lidar_COPDEM_merged.tif')
    if not os.path.exists(merged_dem_fn):
        # construct command 
        cmd = ['gdalwarp', copdem_fill_coreg_fn, lidar_fill_fn, merged_dem_fn]
        out = subprocess.run(cmd, shell=False, capture_output=True)
        print(out)
        
        # plot
        merged_dem = xdem.DEM(merged_dem_fn)
        merged_dem.plot(cmap='terrain')
        plt.show()
    else:
        print('Merged DEM already exists, skipping.')
    

In [ ]:
# Clip MCS lidar to roads
lidar_fn = os.path.join(data_dir, site_name, 'refdem', f'{site_name}_REFDEM_WGS84.tif')
lidar_clip_fn = os.path.splitext(lidar_fn)[0] + '_ROADS-ONLY.tif'
if not os.path.exists(lidar_clip_fn):

    import geopandas as gpd
    import rioxarray as rxr
    import xarray as xr

    site_name = 'MCS'
    roads_fn = os.path.join(data_dir, '..', 'ITD_Functional_Class', 'ITD_HWY_21.shp')

    lidar = rxr.open_rasterio(lidar_fn).squeeze()
    crs = lidar.rio.crs

    roads = gpd.read_file(roads_fn)
    roads = roads.to_crs(crs)
    roads_buffer = roads.geometry[0].buffer(5)

    lidar_clip = lidar.rio.clip([roads_buffer])

    lidar_clip = lidar_clip.rio.write_crs(crs)
    
    lidar_clip.rio.to_raster(lidar_clip_fn)
    print('Lidar DEM clipped to roads and saved to file:', lidar_clip_fn)
    
    